<a href="https://colab.research.google.com/github/Selva-Subramanian/FaceRecognition-DrowsinessDetection/blob/main/Face_Recognition_and_Drowsiness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b> Project Introduction

#### The Indian education landscape has been undergoing rapid changes for the past 10 years owing to the advancement of web-based learning services, specifically, eLearning platforms.
#### Global E-learning is estimated to witness an 8X over the next 5 years to reach USD 2B in 2021. India is expected to grow with a CAGR of 44% crossing the 10M users mark in 2021. Although the market is growing on a rapid scale, there are major challenges associated with digital learning when compared with brick and mortar classrooms. One of many challenges is how to ensure quality learning for students. Digital platforms might overpower physical classrooms in terms of content quality but when it comes to understanding whether students are able to grasp the content in a live class scenario is yet an open-end challenge.
#### In a physical classroom during a lecturing teacher can see the faces and assess the emotion of the class and tune their lecture accordingly, whether he is going fast or slow. He can identify students who need special attention. Digital classrooms are conducted via video telephony software program (ex-Zoom) where it’s not possible for medium scale class (25-50) to see all students and access the mood. Because of this drawback, students are not focusing on content due to lack of surveillance.
#### While digital platforms have limitations in terms of physical surveillance but it comes with the power of data and machines which can work for you. It provides data in the form of video, audio, and texts which can be analysed using deep learning algorithms. Deep learning backed system not only solves the surveillance issue, but it also removes the human bias from the system, and all information is no longer in the teacher’s brain rather translated in numbers that can be analysed and tracked.

# <b> Problem Statement

#### We will solve the above-mentioned challenge by applying deep learning algorithms to live video data. The solution to this problem is by recognizing the face, mark the attendance, log the individual’s session time and put a drowsiness alert.

#### **Face Recognition**
####This is a few shot learning live face attendance systems. The model should be able to real-time identify attending students in the live class based on few images of students.

#### **Drowsiness Detector**
#### This will detect facial landmarks and extract the eye regions. The algorithm should be able to identify students who are not attentive and drowsing in the class.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install tensorflow tensorflow-gpu opencv-python 

     |██████▊                         | 104.6 MB 1.5 MB/s eta 0:04:30ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 128, in resolve
    requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 473, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line

In [ ]:
import cv2
import os 
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/lfw.tgz (Unzipped Files)/lfw'):
    for file in os.listdir(os.path.join('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/lfw.tgz (Unzipped Files)/lfw', directory)):
        EX_PATH = os.path.join('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/lfw.tgz (Unzipped Files)/lfw', directory, file)
        NEW_PATH = os.path.join('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/Negative', file)
        os.replace(EX_PATH, NEW_PATH)

In [ ]:
# Use uuid library to generate unique image names in Anchor
import uuid
os.path.join('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/Anchor', '{}.jpg'.format(uuid.uuid1()))

'/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/Anchor/b3a23aa0-a211-11ec-ae50-0242ac1c0002.jpg'

In [ ]:
# Establish a connection to the webcam
cap = cv2.VideoCapture()
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/Anchor', '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join('/content/drive/MyDrive/Colab Notebooks/AlmaBetter Notebooks/Capstone projects/Face Recognition and Drowsiness Detection - Selva Subramanian S/Dataset/Positive', '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()